In [34]:
import snowflake.connector
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from imblearn.under_sampling import RandomUnderSampler

In [2]:
# connect to snowflake
conn = snowflake.connector.connect(
user='Vance',
password='Meanfatbear1945!',
role = 'ANALYTICS_TEAM',
account= 'vl37350',
warehouse = 'READONLY_COMPUTE',
database = 'ANALYTICS',
schema = 'MARKETING'
)

In [3]:
cur = conn.cursor()
c = '''
SELECT
    *, CASE WHEN RESPONSE_DATE IS NOT NULL THEN 1 ELSE 0 END AS RESPONSE
FROM
    (
    SELECT BIN, MAIL_DATE, MAIL_DROP_MONTH, ESTIMATED_NUMBER_OF_EMPLOYEES, ESTIMATED_ANNUAL_SALES_AMOUNT, BUSINESS_TYPE,  
CASE WHEN TTC038 = 0 AND ACC001 = 0 THEN -1 ELSE ACC001 END AS ACC001, 
CASE WHEN ATC020 <= 0  AND ATC022 = 0  THEN  -1 ELSE ATC022 END AS ATC022, 
CASE WHEN ATC020 <= 0  AND ATC032 = 0  THEN  -1 ELSE ATC032 END AS ATC032,
CASE WHEN ATC020 <= 0  AND ATD034 = 0  THEN  -1 ELSE ATD034 END AS ATD034,
CASE WHEN TTC038 = 0 AND ATP038 = 0 THEN -1 ELSE ATP038 END AS ATP038,
CLO015, CLP019, CTC034, CTD038, IQC001, IQC002, IQC003,
CASE WHEN TTC038=0 AND OTP017 =0 THEN -1 ELSE OTP017 END AS OTP017,
PRO002, RTC042, RTO079,
CASE WHEN TTC038 = 0 AND RTP083 = 0 THEN -1 ELSE RTP083 END AS RTP083,
CASE WHEN TTC038 = 0 AND TTC036 = 0 THEN -1 ELSE TTC036 END AS TTC036,
CASE WHEN TTC038 = 0 AND TTO076 = 0 THEN -1 ELSE TTO076 END AS TTO076,
CASE WHEN TTC038 = 0 AND TTP079 = 0 THEN -1 ELSE TTP079 END AS TTP079,
CASE WHEN TTC038 = 0 AND ACC008 = 0 THEN -1 ELSE ACC008 END AS ACC008,
CASE WHEN ATC020 <= 0 AND ATB001 = 0 THEN -1 ELSE ATB001 END AS ATB001,
CASE WHEN ATC020 <= 0 AND ATB006 = 0 THEN -1 ELSE ATB006 END AS ATB006,
CASE WHEN ATC020 <= 0 AND ATB018 = 0 THEN -1 ELSE ATB018 END AS ATB018,
CASE WHEN ATC020 <= 0 AND ATB019 = 0 THEN -1 ELSE ATB019 END AS ATB019,
CASE WHEN ATC020 <= 0 AND ATC020 = 0 THEN -1 ELSE ATC020 END AS ATC020,
CASE WHEN TTC038 = 0 AND ATP037 = 0 THEN -1 ELSE ATP037 END AS ATP037,
BRC010,
CASE WHEN BRC010 = 0 AND BRO016 = 0 THEN -1 ELSE BRO016 END AS BRO016,
CLB001, CLB002, CLO014, CTB001, CTB006,
CASE WHEN CTC022 <= 0 AND CTB007 = 0 THEN -1 ELSE CTB007 END AS CTB007,
CTB018, CTC022, CTP039, CTP040, CTP041, LGC004,
CASE WHEN TTC038 = 0 AND LSP034 = 0 THEN -1 ELSE LSP034 END AS LSP034,
CASE WHEN TTC038 = 0 AND NTC020 = 0 THEN -1 ELSE NTC020 END AS NTC020,
CASE WHEN NTC020 <= 0 AND NTC021 = 0 THEN -1 ELSE NTC021 END AS NTC021,
CASE WHEN NTC020 <= 0 AND NTD034 = 0 THEN -1 ELSE NTD034 END AS NTD034,
CASE WHEN TTC038 = 0 AND NTP037 = 0 THEN -1 ELSE NTP037 END AS NTP037,
OTB002, OTB005, OTC010,
CASE WHEN TTC038 = 0 AND OTP016 = 0 THEN -1 ELSE OTP016 END AS OTP016,
RTB001, RTB018, RTB026, RTB031, RTB032, RTB034, RTC037, RTC049, RTD067, RTD068, RTD071, RTD072, RTP081,
CASE WHEN TTC038 = 0 AND RTP082 = 0 THEN -1 ELSE RTP082 END AS RTP082,
CASE WHEN TTC036 <= 0 AND TTB002 = 0 THEN -1 ELSE TTB002 END AS TTB002,
CASE WHEN TTC038 = 0 AND TTB003 = 0 THEN -1 ELSE TTB003 END AS TTB003,
CASE WHEN TTC038 = 0 AND TTB004 = 0 THEN -1 ELSE TTB004 END AS TTB004,
CASE WHEN TTC038 = 0 AND TTB008 = 0 THEN -1 ELSE TTB008 END AS TTB008,
CASE WHEN TTC039 = 0 AND TTB009 = 0 THEN -1 ELSE TTB009 END AS TTB009,
TTB015,
CASE WHEN TTC038 = 0 AND TTB020 = 0 THEN -1 ELSE TTB020 END AS TTB020,
CASE WHEN TTC038 = 0 AND TTB023 = 0 THEN -1 ELSE TTB023 END AS TTB023,
CASE WHEN TTC038 = 0 AND TTB033 = 0 THEN -1 ELSE TTB033 END AS TTB033,
TTC038,
CASE WHEN TTC038 = 0 AND TTC039 = 0 THEN -1 ELSE TTC039 END AS TTC039,
CASE WHEN TTC038 = 0 AND TTC044 = 0 THEN -1 ELSE TTC044 END AS TTC044,
CASE WHEN TTC038 = 0 AND TTC051 = 0 THEN -1 ELSE TTC051 END AS TTC051,
CASE WHEN TTC038 = 0 AND TTC052 = 0 THEN -1 ELSE TTC052 END AS TTC052,
CASE WHEN TTC038 = 0 AND TTC054 = 0 THEN -1 ELSE TTC054 END AS TTC054,
CASE WHEN TTC038 = 0 AND TTC055 = 0 THEN -1 ELSE TTC055 END AS TTC055,
CASE WHEN TTC038 = 0 AND TTC056 = 0 THEN -1 ELSE TTC056 END AS TTC056,
CASE WHEN TTC038 = 0 AND TTC067 = 0 THEN -1 ELSE TTC067 END AS TTC067,
CASE WHEN TTC038 = 0 AND TTC068 = 0 THEN -1 ELSE TTC068 END AS TTC068,
CASE WHEN TTC038 = 0 AND TTC069 = 0 THEN -1 ELSE TTC069 END AS TTC069,
CASE WHEN TTC038 = 0 AND TTO074 = 0 THEN -1 ELSE TTO074 END AS TTO074,
CASE WHEN TTC038 = 0 AND TTO075 = 0 THEN -1 ELSE TTO075 END AS TTO075,
CASE WHEN TTC038 = 0 AND TTO077 = 0 THEN -1 ELSE TTO077 END AS TTO077,
CASE WHEN TTC038 = 0 AND TTP080 = 0 THEN -1 ELSE TTP080 END AS TTP080,
CASE WHEN TTC038 = 0 AND TTP084 = 0 THEN -1 ELSE TTP084 END AS TTP084,
TXB004, TXC010,
CASE WHEN TXC010 = 0 AND TXO013 = 0 THEN -1 ELSE TXO013 END AS TXO013,
UCC001, UCC005, YIB, COTTAGE_DUMMY, HOMEBASED_DUMMY, MULTI_LOC, NEW_INQ, BIZAGGZERO, BIZAGG, ANY_BIZAGG   
FROM ANALYTICS.MARKETING.DEP_MASTER_FILE_WSCORES) W

INNER JOIN (SELECT ACTIVATION_ID, THIRDPARTY_RECORD_ID, REPLACE(CAMPAIGN_MONTH, '-', '') AS CAMPAIGN_MONTH FROM ANALYTICS.MARKETING.VW_MAILED_FILE) M
    ON W.BIN = M.THIRDPARTY_RECORD_ID AND W.MAIL_DROP_MONTH = CAMPAIGN_MONTH

LEFT JOIN (SELECT ACTIVATION_CODE_ID, ACTIVATION_METHOD, RESPONSE_DATE FROM ANALYTICS.MARKETING.VW_TARGET_FILE) T
    ON M.ACTIVATION_ID = T.ACTIVATION_CODE_ID
WHERE MAIL_DROP_MONTH BETWEEN '202206' AND '202406' 
AND ESTIMATED_NUMBER_OF_EMPLOYEES != 0 
AND ESTIMATED_ANNUAL_SALES_AMOUNT != 0 
AND BUSINESS_TYPE IS NOT NULL

'''
try: 
    cur.execute(c)
    all_rows = cur.fetch_pandas_all()
    field_names = [i[0] for i in cur.description]
finally:
    cur.close()
response = pd.DataFrame(all_rows)
response.columns = field_names

sub_type = response['ACTIVATION_METHOD']
res = response.drop(['BIN', 'MAIL_DATE', 'MAIL_DROP_MONTH', 'BUSINESS_TYPE', 'THIRDPARTY_RECORD_ID', 
                     'CAMPAIGN_MONTH', 'ACTIVATION_CODE_ID', 'ACTIVATION_ID', 'ACTIVATION_METHOD', 'RESPONSE_DATE'], axis=1)


In [4]:
def calculate_vif_fast(df):
    """Calculate VIF for all columns in a DataFrame."""
    # Calculate the correlation matrix
    corr_matrix = df.corr()
    
    # Compute the VIF for each feature
    vif_data = pd.DataFrame()
    vif_data["Variable"] = df.columns
    vif_data["VIF"] = np.linalg.inv(corr_matrix.values).diagonal()
    
    return vif_data

def drop_high_vif_columns_fast(df, threshold=5.0):
    vif_data = calculate_vif_fast(df)
    while vif_data["VIF"].max() > threshold:
        # Find the column with the highest VIF
        drop_column = vif_data.loc[vif_data["VIF"].idxmax(), "Variable"]
        print(f"Dropping '{drop_column}' with VIF: {vif_data['VIF'].max()}")
        
        # Drop the column and recalculate VIF for the remaining columns
        df = df.drop(columns=[drop_column])
        vif_data = calculate_vif_fast(df)
    
    return df

# Example usage with a large DataFrame
# df = pd.read_csv('your_large_dataframe.csv')  # Load your DataFrame
res_reduced = drop_high_vif_columns_fast(res)

print("Remaining columns after dropping multicollinear ones:")
print(res_reduced.columns)

Dropping 'TTB003' with VIF: 5278565575106.795
Dropping 'CTP040' with VIF: 694245.2233048679
Dropping 'ATB001' with VIF: 88308.69185189666
Dropping 'RTC037' with VIF: 4895.545132223732
Dropping 'TTB008' with VIF: 1906.9158341171528
Dropping 'CTB018' with VIF: 1505.297890975352
Dropping 'TTC039' with VIF: 694.5910870067905
Dropping 'ATB006' with VIF: 541.1695387362339
Dropping 'TTB020' with VIF: 454.4131728730478
Dropping 'LSP034' with VIF: 438.994356625431
Dropping 'CTB006' with VIF: 370.74940843093685
Dropping 'RTC042' with VIF: 301.6003938644298
Dropping 'TTC038' with VIF: 266.6479608445511
Dropping 'TTB002' with VIF: 182.24307093591963
Dropping 'CTP041' with VIF: 141.00778784403968
Dropping 'TTC044' with VIF: 126.71606447582093
Dropping 'TTB023' with VIF: 113.39592558747007
Dropping 'ACC001' with VIF: 107.99377761492948
Dropping 'TTB009' with VIF: 79.30794591514965
Dropping 'LGC004' with VIF: 75.08491230549285
Dropping 'ACC008' with VIF: 72.67010575879938
Dropping 'CTB001' with VIF: 

In [5]:
# # false zero columns
# bizagg_var = ['ACC001', 'ATC022', 'ATC022', 'ATC032', 'ATD034', 
#               'ATP038', 'OTP017', 'RTP083', 'TTC036', 'TTO076', 
#               'TTP079', 'ACC008', 'ATB001', 'ATB006', 'ATB018', 
#               'ATB019', 'ATC020', 'ATP037', 'BRO016', 'CTB007',
#               'LSP034', 'NTC020', 'NTC021', 'NTD034', 'NTP037',
#               'OTP016', 'RTP082', 'TTB002', 'TTB003', 'TTB004',
#               'TTB008', 'TTB009', 'TTB020', 'TTB023', 'TTB033',
#               'TTC039', 'TTC044', 'TTC051', 'TTC052', 'TTC054',
#               'TTC055', 'TTC056', 'TTC067', 'TTC068', 'TTC069',
#               'TTO074', 'TTO075', 'TTO077', 'TTP080', 'TTP084',
#               'TXO013']



In [84]:
def bin_and_woe(df, col_list, target, bins=4):
    woe_dict = {}
    iv_dict = {}
    binned_df = df.copy()
    
    for col in col_list:

        # Binning the column using pd.cut
        binned_df[f'{col}_BINNED'] = pd.cut(binned_df[col], bins=bins, include_lowest=True, labels=False)

        # Convert the binned column to a categorical type
        binned_df[f'{col}_BINNED'] = binned_df[f'{col}_BINNED'].astype('category')

        # Add the new category for zero values and -1's
        negative_bin_value = -2
        zero_bin_value = -1
        binned_df[f'{col}_BINNED'] = binned_df[f'{col}_BINNED'].cat.add_categories([zero_bin_value, negative_bin_value])
        
        # Handle zeros by placing them in the zero bin
        zero_bin = binned_df[binned_df[col] == 0].index
        negative_bin = binned_df[binned_df[col] == -1].index
        binned_df.loc[zero_bin, f'{col}_BINNED'] = zero_bin_value
        binned_df.loc[negative_bin, f'{col}_BINNED'] = negative_bin_value

        # Calculating WOE for each bin
        woe_df = binned_df.groupby(f'{col}_BINNED', observed=False).agg(
            good=(target, lambda x: (x == 0).sum()),
            bad=(target, lambda x: (x == 1).sum())
        )

        woe_df['total'] = woe_df['good'] + woe_df['bad']
        woe_df['good_dist'] = woe_df['good'] / woe_df['good'].sum()
        woe_df['bad_dist'] = woe_df['bad'] / woe_df['bad'].sum()
        
        # Avoiding division by zero
        woe_df['WOE'] = np.log((woe_df['good_dist'] + 1e-5) / (woe_df['bad_dist'] + 1e-5))
        woe_df['IV'] = (woe_df['good_dist'] - woe_df['bad_dist']) * woe_df['WOE']

        # Store WOE and IV for the column
        woe_dict[col] = woe_df['WOE'].to_dict()
        iv_dict[col] = woe_df['IV'].sum()

        # Apply WOE transformation to the binned column
        binned_df[f'{col}_WOE'] = binned_df[f'{col}_BINNED'].map(woe_dict[col])


    return binned_df

# Columns to bin and calculate WOE
col_list = ['ESTIMATED_NUMBER_OF_EMPLOYEES', 'ESTIMATED_ANNUAL_SALES_AMOUNT',
       'ATC022', 'ATD034', 'CLO015', 'CLP019', 'CTC034', 'IQC001', 'IQC003',
       'OTP017', 'PRO002', 'RTO079', 'RTP083', 'TTP079', 'ATB018', 'ATB019',
       'ATP037', 'BRC010', 'BRO016', 'CLB001', 'CLO014', 'CTB007', 'NTC021',
       'NTD034', 'NTP037', 'OTB005', 'OTC010', 'OTP016', 'RTB018', 'RTB026',
       'RTB034', 'RTC049', 'RTD068', 'RTD072', 'RTP081', 'RTP082', 'TTB004',
       'TTB015', 'TTC055', 'TTC067', 'TTC068', 'TTC069', 'TTO074', 'TTO075',
       'TTO077', 'TTP084', 'TXB004', 'TXC010', 'TXO013', 'UCC001', 'UCC005',
       'YIB']

# Target variable
target = 'RESPONSE'

# Bin and calculate WOE
updated_df = bin_and_woe(res_reduced, col_list, target)

# Display the DataFrame with WOE-transformed columns
updated_df

,ESTIMATED_NUMBER_OF_EMPLOYEES,ESTIMATED_ANNUAL_SALES_AMOUNT,ATC022,ATD034,CLO015,CLP019,CTC034,IQC001,IQC003,OTP017,...,TXC010_BINNED,TXC010_WOE,TXO013_BINNED,TXO013_WOE,UCC001_BINNED,UCC001_WOE,UCC005_BINNED,UCC005_WOE,YIB_BINNED,YIB_WOE
0,2,370,0,0,0,0,0,0,0,0,...,-1,-0.009682,-2,-0.009682,0,0.447168,0,0.447424,0,-0.034375
1,1,294,-1,-1,0,0,1,0,0,0,...,0,0.102986,3,0.285416,0,0.447168,0,0.447424,0,-0.034375
2,2,218,-1,-1,0,0,1,0,0,0,...,-1,-0.009682,-2,-0.009682,-1,-0.194071,-1,-0.193392,0,-0.034375
3,2,264,-1,-1,0,0,0,1,0,-1,...,-1,-0.009682,-2,-0.009682,-1,-0.194071,-1,-0.193392,0,-0.034375
4,3,191,-1,-1,0,0,0,1,0,100,...,-1,-0.009682,-2,-0.009682,-1,-0.194071,-1,-0.193392,0,-0.034375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354568,2,81,0,45,0,0,0,0,0,40,...,-1,-0.009682,-2,-0.009682,-1,-0.194071,-1,-0.193392,0,-0.034375
354569,1,158,-1,-1,0,0,0,3,0,0,...,-1,-0.009682,-2,-0.009682,-1,-0.194071,-1,-0.193392,0,-0.034375
354570,4,455,-1,-1,0,0,0,2,0,-1,...,-1,-0.009682,-2,-0.009682,-1,-0.194071,-1,-0.193392,0,-0.034375
354571,2,96,-1,-1,0,0,0,1,1,-1,...,-1,-0.009682,-2,-0.009682,-1,-0.194071,-1,-0.193392,0,-0.034375


In [78]:
target = res['RESPONSE']
final_df = updated_df.drop(['ESTIMATED_NUMBER_OF_EMPLOYEES', 'ESTIMATED_ANNUAL_SALES_AMOUNT',
       'ATC022', 'ATD034', 'CLO015', 'CLP019', 'CTC034', 'IQC001', 'IQC003',
       'OTP017', 'PRO002', 'RTO079', 'RTP083', 'TTP079', 'ATB018', 'ATB019',
       'ATP037', 'BRC010', 'BRO016', 'CLB001', 'CLO014', 'CTB007', 'NTC021',
       'NTD034', 'NTP037', 'OTB005', 'OTC010', 'OTP016', 'RTB018', 'RTB026',
       'RTB034', 'RTC049', 'RTD068', 'RTD072', 'RTP081', 'RTP082', 'TTB004',
       'TTB015', 'TTC055', 'TTC067', 'TTC068', 'TTC069', 'TTO074', 'TTO075',
       'TTO077', 'TTP084', 'TXB004', 'TXC010', 'TXO013', 'UCC001', 'UCC005',
       'YIB', 'ESTIMATED_NUMBER_OF_EMPLOYEES_BINNED', 'ESTIMATED_ANNUAL_SALES_AMOUNT_BINNED',
       'ATC022_BINNED', 'ATD034_BINNED', 'CLO015_BINNED', 'CLP019_BINNED', 'CTC034_BINNED', 'IQC001_BINNED', 'IQC003_BINNED',
       'OTP017_BINNED', 'PRO002_BINNED', 'RTO079_BINNED', 'RTP083_BINNED', 'TTP079_BINNED', 'ATB018_BINNED', 'ATB019_BINNED',
       'ATP037_BINNED', 'BRC010_BINNED', 'BRO016_BINNED', 'CLB001_BINNED', 'CLO014_BINNED', 'CTB007_BINNED', 'NTC021_BINNED',
       'NTD034_BINNED', 'NTP037_BINNED', 'OTB005_BINNED', 'OTC010_BINNED', 'OTP016_BINNED', 'RTB018_BINNED', 'RTB026_BINNED',
       'RTB034_BINNED', 'RTC049_BINNED', 'RTD068_BINNED', 'RTD072_BINNED', 'RTP081_BINNED', 'RTP082_BINNED', 'TTB004_BINNED',
       'TTB015_BINNED', 'TTC055_BINNED', 'TTC067_BINNED', 'TTC068_BINNED', 'TTC069_BINNED', 'TTO074_BINNED', 'TTO075_BINNED',
       'TTO077_BINNED', 'TTP084_BINNED', 'TXB004_BINNED', 'TXC010_BINNED', 'TXO013_BINNED', 'UCC001_BINNED', 'UCC005_BINNED',
       'YIB_BINNED', 'RESPONSE', 'COTTAGE_DUMMY', 'HOMEBASED_DUMMY', 'MULTI_LOC', 'NEW_INQ', 'BIZAGGZERO', 'BIZAGG', 'ANY_BIZAGG'], axis=1)
final_df


,ESTIMATED_NUMBER_OF_EMPLOYEES_WOE,ESTIMATED_ANNUAL_SALES_AMOUNT_WOE,ATC022_WOE,ATD034_WOE,CLO015_WOE,CLP019_WOE,CTC034_WOE,IQC001_WOE,IQC003_WOE,OTP017_WOE,...,TTO074_WOE,TTO075_WOE,TTO077_WOE,TTP084_WOE,TXB004_WOE,TXC010_WOE,TXO013_WOE,UCC001_WOE,UCC005_WOE,YIB_WOE
0,0.000164,-0.000349,0.180520,0.217661,0.011236,0.01326,0.007756,-0.011491,-0.004336,0.005016,...,0.193850,0.182884,-0.158073,-0.184087,-0.009366,-0.009682,-0.009682,0.447168,0.447424,-0.034375
1,0.000164,-0.000349,-0.068124,-0.066173,0.011236,0.01326,-0.141899,-0.011491,-0.004336,0.005016,...,0.193850,0.149303,0.416780,0.279049,0.101734,0.102986,0.285416,0.447168,0.447424,-0.034375
2,0.000164,-0.000349,-0.068124,-0.066173,0.011236,0.01326,-0.141899,-0.011491,-0.004336,0.005016,...,0.193850,0.149303,-0.158073,0.279049,-0.009366,-0.009682,-0.009682,-0.194071,-0.193392,-0.034375
3,0.000164,-0.000349,-0.068124,-0.066173,0.011236,0.01326,0.007756,0.015134,-0.004336,-0.206207,...,-0.206207,-0.206207,-0.158073,-0.206207,-0.009366,-0.009682,-0.009682,-0.194071,-0.193392,-0.034375
4,0.000164,-0.000349,-0.068124,-0.066173,0.011236,0.01326,0.007756,0.015134,-0.004336,0.357833,...,0.024877,0.149303,-0.158073,-0.184087,-0.009366,-0.009682,-0.009682,-0.194071,-0.193392,-0.034375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354568,0.000164,-0.000349,0.180520,-0.133958,0.011236,0.01326,0.007756,-0.011491,-0.004336,0.231213,...,0.024877,0.182884,-0.158073,0.279049,-0.009366,-0.009682,-0.009682,-0.194071,-0.193392,-0.034375
354569,0.000164,-0.000349,-0.068124,-0.066173,0.011236,0.01326,0.007756,0.015134,-0.004336,0.005016,...,0.024877,-0.192617,-0.158073,-0.184087,-0.009366,-0.009682,-0.009682,-0.194071,-0.193392,-0.034375
354570,0.000164,-0.000349,-0.068124,-0.066173,0.011236,0.01326,0.007756,0.015134,-0.004336,-0.206207,...,-0.206207,-0.206207,0.722358,-0.206207,-0.009366,-0.009682,-0.009682,-0.194071,-0.193392,-0.034375
354571,0.000164,-0.000349,-0.068124,-0.066173,0.011236,0.01326,0.007756,0.015134,0.019372,-0.206207,...,-0.206207,-0.206207,-0.158073,-0.206207,-0.009366,-0.009682,-0.009682,-0.194071,-0.193392,-0.034375


In [83]:
y = target
X = final_df

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

forest = RandomForestClassifier(class_weight='balanced', random_state=42)
forest.fit(X_train_rus, y_train_rus)

y_pred = forest.predict(X_test)

In [85]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.57      0.72    105026
           1       0.02      0.63      0.04      1346

    accuracy                           0.57    106372
   macro avg       0.51      0.60      0.38    106372
weighted avg       0.98      0.57      0.72    106372

